In [2]:
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import statsmodels.api as sm
import numpy as np
from scipy.stats import t, norm
from IPython.core.debugger import Pdb
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import requests
import zipfile
import io
import pdb


In [3]:
# Create the data
X = np.array([5, 5, 7])
y = np.array([6, 8, 10])

X_star = np.array([[1, 3],
                   [1, 5]])

normal_returns = np.array([-0.5 + 1.5 * 3, -0.5 + 1.5 * 5])

event_window_returns = np.array([12, 16])



# Add constant for intercept
X = sm.add_constant(X)
print(X)


# Fit the model
model = sm.OLS(y, X).fit()



[[1. 5.]
 [1. 5.]
 [1. 7.]]


In [4]:
# Print the summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.500
Method:                 Least Squares   F-statistic:                     3.000
Date:                Thu, 31 Oct 2024   Prob (F-statistic):              0.333
Time:                        13:32:40   Log-Likelihood:                -3.6486
No. Observations:                   3   AIC:                             11.30
Df Residuals:                       1   BIC:                             9.494
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5000      4.975     -0.101      0.9

/Users/nicolasroever/opt/anaconda3/envs/debt_crisis/lib/python3.11/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [5]:
# Get fitted values (ŷ)
y_fitted = model.fittedvalues

# Get residuals (errors)
residuals = model.resid


# Print results
print("Actual values (y):")
print(y)
print("\nFitted values (ŷ):")
print(y_fitted)
print("\nResiduals (e = y - ŷ):")
print(residuals)

Actual values (y):
[ 6  8 10]

Fitted values (ŷ):
[ 7.  7. 10.]

Residuals (e = y - ŷ):
[-1.  1.  0.]


In [6]:
dot_product = np.dot(residuals, residuals)
print(dot_product)

error_variance = dot_product / (len(y) - 2)
print(error_variance)


2.0
2.0


In [7]:
# Calculate X transpose
X_transpose = X.T
print(X_transpose)

# Calculate X'X
XTX = np.dot(X_transpose, X)
print(XTX)

# Calculate inverse of X'X
XTX_inv = np.linalg.inv(XTX)
print(XTX_inv)

[[1. 1. 1.]
 [5. 5. 7.]]
[[ 3. 17.]
 [17. 99.]]
[[12.375 -2.125]
 [-2.125  0.375]]


In [10]:
error_variance

2.0

In [11]:
#Now, I want to get the variance estimator for abnormal returns during event window.

X_star_transpose = X_star.T

X_star_XTX_inv_X_star_transpose = np.dot(np.dot(X_star, XTX_inv), X_star.T)

print(X_star_XTX_inv_X_star_transpose)

variance = np.eye(2) * error_variance + X_star_XTX_inv_X_star_transpose * error_variance

print(variance)

[[3.  1. ]
 [1.  0.5]]
[[8. 2.]
 [2. 3.]]


In [9]:
normal_returns



array([4., 7.])

In [32]:



abnormal_returns = event_window_returns - normal_returns

print(abnormal_returns)



[8. 9.]


In [33]:
CAR = np.sum(abnormal_returns)

print(CAR)



17.0


In [34]:

# Calculate the product
VAR_CAR = np.ones(2) @ variance @ np.ones(2).T
# or alternatively: VAR_CAR = ones @ variance @ ones.T

print("Result of [1 1] × variance × [1 1]ᵀ:")
print(VAR_CAR)

Result of [1 1] × variance × [1 1]ᵀ:
223.0000000000007
